In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [257]:
test = pd.read_csv("test_transformed_allFeatures.csv")
df = pd.read_csv("train_transformed_allFeat.csv")
#test.drop(test.columns[[13,14]], axis=1, inplace=True)
#df.drop(df.columns[[13,14]], axis=1, inplace=True)

In [240]:
df.iloc[:,4:]

,aa,ra,jc,pa,path_len,dc_A,dc_B,cc_A,cc_B,hits_A,hits_B,bet_cA,bet_cB
0,2.471649,0.531746,0.428571,96,1,0.002124,0.003186,0.187879,0.223134,0.000003,5.710000e-05,0.000004,7.627580e-04
1,2.976054,0.668651,0.466667,112,1,0.002124,0.003717,0.187879,0.209683,0.000003,1.070000e-05,0.000004,7.945610e-04
2,2.899858,0.628968,0.700000,72,1,0.002124,0.002390,0.187879,0.187891,0.000003,3.200000e-06,0.000004,9.620000e-06
3,2.471649,0.531746,0.600000,64,1,0.002124,0.002124,0.187879,0.181356,0.000003,2.090000e-06,0.000004,4.700000e-08
4,2.976054,0.668651,0.466667,112,1,0.002124,0.003717,0.187879,0.208769,0.000003,1.830000e-05,0.000004,4.391450e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32067,0.000000,0.000000,0.000000,510,3,0.002655,0.001328,0.246668,0.225652,0.000229,4.054780e-04,0.000371,4.349246e-03
32068,0.000000,0.000000,0.000000,30,999,0.002655,0.001593,0.246668,0.173938,0.000229,5.740000e-07,0.000371,0.000000e+00
32069,0.000000,0.000000,0.000000,310,3,0.002655,0.002390,0.246668,0.224344,0.000229,3.290000e-05,0.000371,6.463828e-03
32070,0.000000,0.000000,0.000000,140,3,0.002655,0.001062,0.246668,0.213252,0.000229,2.260000e-05,0.000371,3.241458e-03


In [267]:
test.iloc[:,4:13]

,aa,ra,jc,pa,path_len,dc_A,dc_B,cc_A,cc_B
0,0.000000,0.000000,0.000000,56,3,0.002124,0.001859,0.187879,0.218136
1,0.000000,0.000000,0.000000,24,5,0.002124,0.000797,0.187879,0.203260
2,0.000000,0.000000,0.000000,496,3,0.004249,0.008232,0.229441,0.256824
3,1.242670,0.400000,0.080000,72,1,0.006373,0.000797,0.263182,0.206104
4,0.000000,0.000000,0.000000,391,4,0.004514,0.006107,0.247700,0.246366
...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,2,5,0.000266,0.000531,0.177465,0.217868
1996,0.000000,0.000000,0.000000,2,999,0.000531,0.000266,0.228538,0.000266
1997,0.000000,0.000000,0.000000,116,3,0.001062,0.007700,0.214815,0.249028
1998,1.766048,0.424368,0.053333,370,1,0.019649,0.001328,0.280597,0.231677


In [280]:
y = [True if i < df.shape[0]/2 else False for i in range(df.shape[0])]
x = df.iloc[:,4:13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [281]:
collections.Counter(y)

Counter({True: 16036, False: 16036})

In [272]:
#from sklearn.preprocessing import StandardScaler
#normal = StandardScaler()
#x_train = normal.fit_transform(x_train)
#x_test = normal.transform(x_test)

LR = LogisticRegression(n_jobs = 4).fit(x_train, y_train)
LRT = LR.predict(x_test)

print(metrics.classification_report(y_test, LRT))


              precision    recall  f1-score   support

       False       1.00      1.00      1.00      6324
        True       1.00      1.00      1.00      6505

    accuracy                           1.00     12829
   macro avg       1.00      1.00      1.00     12829
weighted avg       1.00      1.00      1.00     12829



In [273]:
probs = LR.predict_proba(x_test)

In [274]:
metrics.roc_auc_score(y_test, probs[:, 1])

1.0

In [276]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:13]))

Counter({True: 1261, False: 739})

In [277]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

ValueError: X has 13 features per sample; expecting 9

In [278]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg.csv', header = True, index= False)

### Handmade gridsearch

In [248]:
import itertools as it
from sklearn.metrics import accuracy_score
LR = LogisticRegression
LR_param = {'penalty':['l1', 'l2', 'elasticnet'],
            'C':[1.0,0.5,0.1,0.01], 'max_iter':[200],
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
models = [('Logistic Regression',LR,LR_param)]

result = []
best = {}
for name, m, params in models:
    keys, values = zip(*params.items())
    permutations = [dict(zip(keys, v)) for v in it.product(*values)]
    
    for p in permutations:
        try:
            mod = m(**p)
            mod.fit(x_train, y_train);
            y_pred = mod.predict(x_test)
            Acc_scored = accuracy_score(y_test, y_pred)
            cnt = collections.Counter(mod.predict(test.iloc[:,4:]))
            best[str(p)] = (Acc_scored, cnt)
            #print('Model: ',name, ' HyperParams: ',p,' Accuracy: ',Acc_scored)
        except:
            pass#print(p)

    #best_params = max(best, key = lambda k:best[k][0])
    #result.append((name, best_params, best[best_params][0], best[best_params][1]))  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

In [249]:
[print(k,v) for k,v in best.items()]

{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'} (1.0, Counter({False: 1229, True: 771}))
{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'saga'} (0.7070699197131499, Counter({False: 1271, True: 729}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'liblinear'} (1.0, Counter({False: 1340, True: 660}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'saga'} (0.7071478681113104, Counter({False: 1271, True: 729}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'liblinear'} (1.0, Counter({False: 1347, True: 653}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'saga'} (0.7070699197131499, Counter({False: 1271, True: 729}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'liblinear'} (1.0, Counter({False: 1342, True: 658}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'saga'} (0.7061345389352249, Counter({False: 1269, True: 731}))
{'penalty': 'l2', 'C': 1.0, 'max_iter': 200, 'solver': 'newton-cg'} (1.0, Counter({Fal

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### LGBM

In [183]:
import lightgbm as lgbm

train_data = lgbm.Dataset(x_train, pd.Series(y_train))
test_data = lgbm.Dataset(x_test, pd.Series(y_test))

# define parameters
parameters = {
    'objective': 'binary',
        'boosting_type': 'gbdt',
#             'boosting_type': 'rf',
        'nthread': 4,
        'learning_rate': 0.02,  # 02,
        'num_leaves': 15,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'early_stopping_round':5,
        'max_depth':2,
        'reg_alpha': 0.041545473,
        'reg_lambda': 0.0735294,
        'min_split_gain': 0.0222415,
        'min_child_weight': 60,
        'seed': 0,
        'verbose': -1,
        'metric': 'auc'
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=20)

[1]	valid_0's auc: 1
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 1
[3]	valid_0's auc: 1
[4]	valid_0's auc: 1
[5]	valid_0's auc: 1
[6]	valid_0's auc: 1
Early stopping, best iteration is:
[1]	valid_0's auc: 1


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [170]:
predict

array([0.48783832, 0.48783832, 0.48783832, ..., 0.48783832, 0.50783456,
       0.48783832])

In [185]:
predict= model.predict(test.iloc[:,7:])
collections.Counter(["pos" if i>0.5 else "neg" for i in predict])

Counter({'neg': 2000})

### Support Vector Machine (SVM)

In [57]:
SVM = LinearSVC(max_iter=10000)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_test)


print(metrics.classification_report(y_test, clf_predict))

C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


              precision    recall  f1-score   support

         neg       1.00      1.00      1.00      4041
         pos       1.00      1.00      1.00      3977

    accuracy                           1.00      8018
   macro avg       1.00      1.00      1.00      8018
weighted avg       1.00      1.00      1.00      8018



C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [58]:
probs = clf.predict_proba(x_test)

In [59]:
probs

array([[0.67533204, 0.32466796],
       [0.34029695, 0.65970305],
       [0.35430718, 0.64569282],
       ...,
       [0.80227792, 0.19772208],
       [0.67984221, 0.32015779],
       [0.67347837, 0.32652163]])

In [60]:
metrics.roc_auc_score(y_test, probs[:, 1])

1.0

In [61]:
collections.Counter(clf.predict(test.iloc[:,4:]))

Counter({'neg': 1465, 'pos': 535})

In [42]:
collections.Counter(y_test)

Counter({'neg': 4029, 'pos': 3989})

In [20]:
results = clf.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.32588784, 0.11201567, 0.32562297, ..., 0.32601893, 0.65587601,
       0.11203549])

In [160]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': predict})
output.to_csv('lightGBM.csv', header = True, index= False)